In [1]:
from src import *
from bokeh.io import output_notebook

In [2]:
output_notebook()

Loading BokehJS ...

In [ ]:
all_potential_countries = [
#     InputConfig("bosnia-and-herzegovina", 1845000),
#     InputConfig("germany", 83020000),
#     InputConfig("france", 66990000),
#     InputConfig("italy", 60360000),
#     InputConfig("portugal", 10280000),
#     InputConfig("spain", 46940000),
#     InputConfig("canada", 14750000, province="Ontario"),
#     InputConfig("greece", 10720000),
#     InputConfig("czech-republic", 10690000),
#     InputConfig("netherlands", 17280000),
#     InputConfig("croatia", 4058000),
#     InputConfig("romania", 19410000),
#     InputConfig("kosovo", 10280000),
#     InputConfig("albania", 2846000),
#     InputConfig("belarus", 9485000),
#     InputConfig("poland", 38000000),
#     InputConfig("slovenia", 2081000),
#     InputConfig("bulgaria", 7000000),
#     InputConfig("cyprus", 1189000),
#     InputConfig("estonia", 1329000),
#     InputConfig("latvia", 1920000),
#     InputConfig("serbia", 6982000),
#     InputConfig("switzerland", 8570000),   
#     InputConfig("ukraine", 42000000),
#     InputConfig("ireland", 5000000),
#     InputConfig("sweden", 10230000),
#     InputConfig("united-states", 328200000),
]

In [10]:
example_configs = [
    InputConfig("germany", 83020000),
    InputConfig("italy", 60360000),
    InputConfig("france", 66990000),
    InputConfig("switzerland", 8570000),
    InputConfig("canada", 14750000, province="Ontario"),
    InputConfig("spain", 46940000),
    InputConfig("france", 66990000),
    InputConfig("greece", 10720000),
]

In [11]:
graph_data(example_configs)

In [42]:
def get_europian_slugs():
    europe_iso2 = [
        'AL', 'AD', 'AM', 'AT', 'BY', 'BE', 'BA', 'BG', 'CH', 'CY', 'CZ', 'DE',
        'DK', 'EE', 'ES', 'FO', 'FI', 'FR', 'GB', 'GE', 'GI', 'GR', 'HU', 'HR',
        'IE', 'IS', 'IT', 'LI', 'LT', 'LU', 'LV', 'MC', 'MK', 'MT', 'NO', 'NL', 'PL',
        'PT', 'RO', 'RU', 'SE', 'SI', 'SK', 'SM', 'TR', 'UA', 'VA',
    ]
    result = []
    
    for country_dict in get_countries():
        if country_dict['ISO2'] in europe_iso2:
            result.append(country_dict['Slug'])
    
    return result

In [43]:
slugs = get_europian_slugs()

[   {'Country': 'Kiribati', 'ISO2': 'KI', 'Slug': 'kiribati'},
    {'Country': 'Luxembourg', 'ISO2': 'LU', 'Slug': 'luxembourg'},
    {'Country': 'Mauritania', 'ISO2': 'MR', 'Slug': 'mauritania'},
    {'Country': 'Mauritius', 'ISO2': 'MU', 'Slug': 'mauritius'},
    {'Country': 'Panama', 'ISO2': 'PA', 'Slug': 'panama'},
    {   'Country': 'Venezuela (Bolivarian Republic)',
        'ISO2': 'VE',
        'Slug': 'venezuela'},
    {'Country': 'French Polynesia', 'ISO2': 'PF', 'Slug': 'french-polynesia'},
    {'Country': 'Kyrgyzstan', 'ISO2': 'KG', 'Slug': 'kyrgyzstan'},
    {'Country': 'Montserrat', 'ISO2': 'MS', 'Slug': 'montserrat'},
    {'Country': 'Nicaragua', 'ISO2': 'NI', 'Slug': 'nicaragua'},
    {'Country': 'Norway', 'ISO2': 'NO', 'Slug': 'norway'},
    {'Country': 'Turkmenistan', 'ISO2': 'TM', 'Slug': 'turkmenistan'},
    {'Country': 'Argentina', 'ISO2': 'AR', 'Slug': 'argentina'},
    {'Country': 'Malaysia', 'ISO2': 'MY', 'Slug': 'malaysia'},
    {'Country': 'Pakistan', 'ISO2': '